https://www.kaggle.com/datasets/parthplc/ms-marco-dataset

In [1]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... - \ | done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=83006e28430a6bd25fb877608047d798093b681160dde439d02a7a5a644dc673
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0

In [2]:
import os
import pandas as pd
from tqdm import tqdm
from googletrans import Translator
import pandas as pd
import time
tqdm.pandas()

translator = Translator()


def translate_batch(texts, src='en', dest='hr', wait_time=5):
    def translate_one(text):
        retries = 0
        while True:
            try:
                return translator.translate(text, src=src, dest=dest).text
            except Exception as e:
                retries += 1
                print(f"Warning: Translation failed. Retry {retries} in {wait_time} seconds...")
                time.sleep(wait_time)
    return [translate_one(t) for t in texts]

# Function to process a column in batches
def translate_column_in_batches(df, col, batch_size=50, output_dir="translations"):
    os.makedirs(output_dir, exist_ok=True) 
    translated_data = []

    # Split the column into batches
    num_batches = (len(df) + batch_size - 1) // batch_size  # Calculate total batches
    for batch_num in tqdm(range(num_batches), desc=f"Translating {col}"):
        start_idx = batch_num * batch_size
        end_idx = min(start_idx + batch_size, len(df))
        batch = df[col].iloc[start_idx:end_idx]

        # Define batch file name
        batch_file = os.path.join(output_dir, f"{col}_batch_{batch_num}.csv")

        # Check if batch already exists
        if os.path.exists(batch_file):
            print(f"Loading batch {batch_num} from file: {batch_file}")
            batch_translated = pd.read_csv(batch_file)['translated'].tolist()
        else:
            # print(f"Translating batch {batch_num}...")
            batch_translated = translate_batch(batch.tolist())
            # Save the batch to a CSV file
            pd.DataFrame({'original': batch.tolist(), 'translated': batch_translated}).to_csv(batch_file, index=False)

        translated_data.extend(batch_translated)

    return translated_data

In [3]:
input_path = "/kaggle/input/ms-marco/train.csv"
output_path = "/kaggle/working/ms-marco-translated.csv"
batch_size = 10
output_tmp_dir = f"/kaggle/working/translations_{batch_size}"

df = pd.read_csv(input_path)
df = df[:10000]
df

,answers,query,finalpassage
0,"Kids who are bipolar, in their manic stages, v...",why do children get aggressive,"At the same time, despite claiming the review ..."
1,"Equifax, transunion and experian.",which credit bureau is used the most for auto ...,Best Answer: both of those answers are wrong. ...
2,"Women eat at least 1,200 calories daily and me...",what is the minimum healthy calorie intake,Safe Intakes. If you’re not supervised by a me...
3,Because Caffeine increases the stress hormone ...,why is coffee making gain weight,Is coffee making you fat? If you are overweigh...
4,Kent County,"what county is grand rapids, mi in","Located in Grand Rapids, Michigan, the 61st Di..."
...,...,...,...
9995,"Responsible organizers, driven to create and e...",what is a istj,A person with an ISTJ personality is more Intr...
9996,Saturn,which planet is colder saturn or neptune,"Saturn: Due to its distance from the Sun, Satu..."
9997,"A peace treaty between the five nations, Franc...",what were the terms of the treaty of versailles,"The Versailles Treaty, signed on June 28, 1919..."
9998,"The average weather temperature in big creek, ...","average weather temperature in big creek, ca?",Average Weather and Climate in November in Big...


In [4]:
columns_to_translate = ['finalpassage', 'query']

for col in columns_to_translate:
    print(f"Processing column: {col}")
    df[f'{col}_cro'] = translate_column_in_batches(df, col, batch_size=batch_size, output_dir=output_tmp_dir)

df.to_csv(output_path, index=False)
print("Translation completed and saved.")


Processing column: finalpassage


Translating finalpassage: 100%|██████████| 1000/1000 [3:29:53<00:00, 12.59s/it]


Processing column: query


Translating query: 100%|██████████| 1000/1000 [3:23:29<00:00, 12.21s/it]


Translation completed and saved.
